In [1]:
from IPython.display import Markdown, display
def printmd(string):
    display(Markdown('# <span style="color:red">'+string+'</span>'))


if ('sc' in locals() or 'sc' in globals()):
    printmd('<<<<<!!!!! It seems that you are running in a IBM Watson Studio Apache Spark Notebook. Please run it in an IBM Watson Studio Default Runtime (without Apache Spark) !!!!!>>>>>')


In [2]:
!pip install pyspark==2.4.5

     |████████████████████████████████| 217.8 MB 10 kB/s s eta 0:00:01
     |████████████████████████████████| 197 kB 39.2 MB/s eta 0:00:01
  Created wheel for pyspark: filename=pyspark-2.4.5-py2.py3-none-any.whl size=218257927 sha256=6ed5c85483edd83eb85c840564485d86559a62a79499c4189db20f43708e3a8c
  Stored in directory: /tmp/wsuser/.cache/pip/wheels/01/c0/03/1c241c9c482b647d4d99412a98a5c7f87472728ad41ae55e1e
Successfully built pyspark


In [3]:
try:
    from pyspark import SparkContext, SparkConf
    from pyspark.sql import SparkSession
except ImportError as e:
    printmd('<<<<<!!!!! Please restart your kernel after installing Apache Spark !!!!!>>>>>')

In [4]:
sc = SparkContext.getOrCreate(SparkConf().setMaster("local[*]"))

spark = SparkSession \
    .builder \
    .getOrCreate()

Leeremos un DataFrame para realizar un análisis estadístico sencillo. Luego reequilibraremos el conjunto de datos.

Creemos un DataFrame a partir de un archivo remoto:



In [5]:
# borramos archivos de ejecuciones previas
!rm -f hmp.parquet*

# descargamos los datos que contienen los datos en formato PARQUET
!wget https://github.com/IBM/coursera/raw/master/hmp.parquet
    
# creamos un dataframe a partir de los datos
df = spark.read.parquet('hmp.parquet')

# registramos la correspondiente query table
df.createOrReplaceTempView('df')

--2021-02-18 21:54:58--  https://github.com/IBM/coursera/raw/master/hmp.parquet
Resolving github.com (github.com)... 140.82.112.3
Connecting to github.com (github.com)|140.82.112.3|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://github.com/IBM/skillsnetwork/raw/master/hmp.parquet [following]
--2021-02-18 21:54:58--  https://github.com/IBM/skillsnetwork/raw/master/hmp.parquet
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/IBM/skillsnetwork/master/hmp.parquet [following]
--2021-02-18 21:54:58--  https://raw.githubusercontent.com/IBM/skillsnetwork/master/hmp.parquet
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 93299

Miremos primero nuestro dataset. El mismo contiene sensores que registran diferentes actividades de movimiento
X, Y y Z contienen valores de sensor de acelerómetro, mientras que el campo class contiene información sobre qué movimiento se ha registrado.
El campo source es opcional y se puede utilizar para el linaje de datos, ya que contiene el nombre de archivo del archivo original desde donde se importó la fila en particular.

Más detalles pueden encontrarse aquí:
<https://github.com/wchill/HMP_Dataset>


In [6]:
df.show()
df.printSchema()

+---+---+---+--------------------+-----------+
|  x|  y|  z|              source|      class|
+---+---+---+--------------------+-----------+
| 22| 49| 35|Accelerometer-201...|Brush_teeth|
| 22| 49| 35|Accelerometer-201...|Brush_teeth|
| 22| 52| 35|Accelerometer-201...|Brush_teeth|
| 22| 52| 35|Accelerometer-201...|Brush_teeth|
| 21| 52| 34|Accelerometer-201...|Brush_teeth|
| 22| 51| 34|Accelerometer-201...|Brush_teeth|
| 20| 50| 35|Accelerometer-201...|Brush_teeth|
| 22| 52| 34|Accelerometer-201...|Brush_teeth|
| 22| 50| 34|Accelerometer-201...|Brush_teeth|
| 22| 51| 35|Accelerometer-201...|Brush_teeth|
| 21| 51| 33|Accelerometer-201...|Brush_teeth|
| 20| 50| 34|Accelerometer-201...|Brush_teeth|
| 21| 49| 33|Accelerometer-201...|Brush_teeth|
| 21| 49| 33|Accelerometer-201...|Brush_teeth|
| 20| 51| 35|Accelerometer-201...|Brush_teeth|
| 18| 49| 34|Accelerometer-201...|Brush_teeth|
| 19| 48| 34|Accelerometer-201...|Brush_teeth|
| 16| 53| 34|Accelerometer-201...|Brush_teeth|
| 18| 52| 35|

Este es un dataset de clasificación clásico. Algo que siempre hacemos durante el análisis es chequear si las clases están balanceadas; es decir, si hay más o menos del mismo número de ejemplos en cada clase. Averiguemoslo usando SQL.


In [7]:
spark.sql('select class,count(*) from df group by class').show()

+--------------+--------+
|         class|count(1)|
+--------------+--------+
| Use_telephone|   15225|
| Standup_chair|   25417|
|      Eat_meat|   31236|
|     Getup_bed|   45801|
|   Drink_glass|   42792|
|    Pour_water|   41673|
|     Comb_hair|   23504|
|          Walk|   92254|
|  Climb_stairs|   40258|
| Sitdown_chair|   25036|
|   Liedown_bed|   11446|
|Descend_stairs|   15375|
|   Brush_teeth|   29829|
|      Eat_soup|    6683|
+--------------+--------+



Puede verse que hay desbalance entre clases.
Antes de seguir rescribamos la misma consulta usando la DataFrame API. Recuerde que no importa si realiza sus consultas con esta API o con SQL, todo se reduce al mismo plan de ejecucuión, que es optimizado por Tungsten y acelerado por Catalyst.
Más detalles pueden consultarse aquí:
[https://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrame](https://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrame?cm_mmc=Email_Newsletter-_-Developer_Ed%2BTech-_-WW_WW-_-SkillsNetwork-Courses-IBMDeveloperSkillsNetwork-ML0201EN-SkillsNetwork-20647446&cm_mmca1=000026UJ&cm_mmca2=10006555&cm_mmca3=M12345678&cvosrc=email.Newsletter.M12345678&cvo_campaign=000026UJ&cm_mmc=Email_Newsletter-_-Developer_Ed%2BTech-_-WW_WW-_-SkillsNetwork-Courses-IBMDeveloperSkillsNetwork-ML0201EN-SkillsNetwork-20647446&cm_mmca1=000026UJ&cm_mmca2=10006555&cm_mmca3=M12345678&cvosrc=email.Newsletter.M12345678&cvo_campaign=000026UJ)


In [8]:
df.groupBy('class').count().show()

+--------------+-----+
|         class|count|
+--------------+-----+
| Use_telephone|15225|
| Standup_chair|25417|
|      Eat_meat|31236|
|     Getup_bed|45801|
|   Drink_glass|42792|
|    Pour_water|41673|
|     Comb_hair|23504|
|          Walk|92254|
|  Climb_stairs|40258|
| Sitdown_chair|25036|
|   Liedown_bed|11446|
|Descend_stairs|15375|
|   Brush_teeth|29829|
|      Eat_soup| 6683|
+--------------+-----+



Crearemos un diagrama de barras para los datos.Usaremos la librería pixidust, pero también podríamos usar otra, como Matplotlib.



In [11]:
#!pip install pixiedust
import pixiedust
from pyspark.sql.functions import col
counts = df.groupBy('class').count().orderBy('count')
display(counts)

DataFrame[class: string, count: bigint]

Queremos ahora otras métricas del desbalance, como el mínimo, máximo, media y desviación estándar. Si dividimos el máximo por el mínimo obtenemos una medida llamada cociente minmax (minmax ratio) que nos habla de la relación entre las clases más pequeña y más grande.


In [12]:
spark.sql('''
    select 
        *,
        max/min as minmaxratio -- compute minmaxratio based on previously computed values
        from (
            select 
                min(ct) as min, -- compute minimum value of all classes
                max(ct) as max, -- compute maximum value of all classes
                mean(ct) as mean, -- compute mean between all classes
                stddev(ct) as stddev -- compute standard deviation between all classes
                from (
                    select
                        count(*) as ct -- count the number of rows per class and rename it to ct
                        from df -- access the temporary query table called df backed by DataFrame df
                        group by class -- aggrecate over class
                )
        )   
''').show()

+----+-----+------------------+------------------+-----------------+
| min|  max|              mean|            stddev|      minmaxratio|
+----+-----+------------------+------------------+-----------------+
|6683|92254|31894.928571428572|21284.893716741157|13.80427951518779|
+----+-----+------------------+------------------+-----------------+



Realizemos ahora la misma consulta con la DataFrame API.


In [13]:
from pyspark.sql.functions import col, min, max, mean, stddev

df \
    .groupBy('class') \
    .count() \
    .select([ 
        min(col("count")).alias('min'), 
        max(col("count")).alias('max'), 
        mean(col("count")).alias('mean'), 
        stddev(col("count")).alias('stddev') 
    ]) \
    .select([
        col('*'),
        (col("max") / col("min")).alias('minmaxratio')
    ]) \
    .show()


+----+-----+------------------+------------------+-----------------+
| min|  max|              mean|            stddev|      minmaxratio|
+----+-----+------------------+------------------+-----------------+
|6683|92254|31894.928571428572|21284.893716741157|13.80427951518779|
+----+-----+------------------+------------------+-----------------+



Now it’s time for you to work on the data set. First, please create a table of all classes with the respective counts, but this time, please order the table by the count number, ascending.


In [14]:
spark.sql('select class,count(*) from df group by class order by count(*) asc').show()

+--------------+--------+
|         class|count(1)|
+--------------+--------+
|      Eat_soup|    6683|
|   Liedown_bed|   11446|
| Use_telephone|   15225|
|Descend_stairs|   15375|
|     Comb_hair|   23504|
| Sitdown_chair|   25036|
| Standup_chair|   25417|
|   Brush_teeth|   29829|
|      Eat_meat|   31236|
|  Climb_stairs|   40258|
|    Pour_water|   41673|
|   Drink_glass|   42792|
|     Getup_bed|   45801|
|          Walk|   92254|
+--------------+--------+



Modifique el diagrama de barras para que esté ordenado por el número de elementos por clase, en orden ascendente.


In [15]:
from pyspark.sql.functions import col
counts = df.groupBy('class').count().orderBy('count').sort('class')
display(counts)

DataFrame[class: string, count: bigint]

Las clases desbalanceadas pueden ser un dolor de cabeza en Machine Learning. Por tanto hay que rebalancear. En lo que sigue limitamos el número de elementos por clase a la cantidad de la clase menos representativa. Esto se llama undersampling.
Otras formas de rebalancear pueden encontrarse aquí:
[https://machinelearningmastery.com/tactics-to-combat-imbalanced-classes-in-your-machine-learning-dataset/](https://machinelearningmastery.com/tactics-to-combat-imbalanced-classes-in-your-machine-learning-dataset?cm_mmc=Email_Newsletter-_-Developer_Ed%2BTech-_-WW_WW-_-SkillsNetwork-Courses-IBMDeveloperSkillsNetwork-ML0201EN-SkillsNetwork-20647446&cm_mmca1=000026UJ&cm_mmca2=10006555&cm_mmca3=M12345678&cvosrc=email.Newsletter.M12345678&cvo_campaign=000026UJ)


In [16]:
from pyspark.sql.functions import min

# creamos muchas clases distintas a partir del dataset
classes = [row[0] for row in df.select('class').distinct().collect()]

# computamos el número de elementps de la clase más chica para limitar el número de muestras por clase
min = df.groupBy('class').count().select(min('count')).first()[0]

# definimos el dataframe para el resultado
df_balanced = None

# iteramos sobre las distintas clases
for cls in classes:
    
    # solo seleccione ejemplos para la clase específica dentro de esta iteración
    # barajar el orden de los elementos (al establecer la fracción en 1.0, la muestra funciona como barajar)
    # retornamos solamente las primeras n muestras
    df_temp = df \
        .filter("class = '"+cls+"'") \
        .sample(False, 1.0) \
        .limit(min)
    
    # en la primera iteración, asignamos df_temp al df_balanced vacío
    if df_balanced == None:    
        df_balanced = df_temp
    # luego, agregamos verticalmente
    else:
        df_balanced=df_balanced.union(df_temp)

Ahora verifiiquemos que df_balanced tenga el mismo número de elementos por clase.


In [18]:
df_balanced.groupBy('class').count().show()

+--------------+-----+
|         class|count|
+--------------+-----+
| Use_telephone| 6683|
| Standup_chair| 6683|
|      Eat_meat| 6683|
|     Getup_bed| 6683|
|   Drink_glass| 6683|
|    Pour_water| 6683|
|     Comb_hair| 6683|
|          Walk| 6683|
|  Climb_stairs| 6683|
| Sitdown_chair| 6683|
|   Liedown_bed| 6683|
|Descend_stairs| 6683|
|   Brush_teeth| 6683|
|      Eat_soup| 6683|
+--------------+-----+

